In [56]:
import pandas as pd

# Replace the file path with your actual file path
file_path = r"C:\Users\njrav\Downloads\UNO Service Learning Data Sheet De-Identified Version.xlsx"

# Read the Excel file
data = pd.read_excel(file_path)

# Display the first few rows of the data
print(data.head())


   Patient ID# Grant Req Date  App Year  ...  Patient Letter Notified? (Directly/Indirectly through rep) Application Signed? Notes
0       180001     2018-10-17         1  ...                                            Missing                      Missing   NaN
1       190001     2019-01-03         1  ...                                            Missing                      Missing   NaN
2       190001     2019-03-11         1  ...                                            Missing                      Missing   NaN
3       190002     2019-05-20         1  ...                                            Missing                      Missing   NaN
4       190003     2019-05-22         1  ...                                            Missing                      Missing   NaN

[5 rows x 30 columns]


In [ ]:
import re

# Replace variations of 'ck' with 'Check'
data['Payment Method'] = data['Payment Method'].str.replace(r'(?i)ck[^a-zA-Z]*\d{0,4}', 'Check', regex=True)

# Convert values that look like dates into "Yes"
data['Payment Submitted?'] = data['Payment Submitted?'].apply(
    lambda x: 'Yes' if pd.to_datetime(x, errors='coerce') not in [None, pd.NaT] else x
)

def clean_city_name(city):
    if pd.isna(city):  # Handle missing values
        return city

    city = city.strip()  # Remove leading/trailing spaces
    city = re.sub(r'[?,()]', '', city)  # Remove unwanted characters
    
    # Standardize "St. Paul"
    if city.lower() in ["saint paul", "st paul"]:
        return "St. Paul"
    
    return city.title()  # Capitalize first letter of each word

# Apply the function to the column
data['Pt City'] = data['Pt City'].apply(clean_city_name)

# Filling NA City Values with "Missing"
data["Pt City"] = data["Pt City"].fillna("Missing")

# Replace specific states with their abbreviations
data["Pt State"] = data["Pt State"].replace({"Florida": "FL", "Iowa": "IA"})

# Capitalize state abbreviations
data["Pt State"] = data["Pt State"].str.upper()

# Fill missing values with "Missing"
data["Pt State"] = data["Pt State"].fillna("Missing")

# Convert ZIP codes to strings to avoid TypeError
data["Pt Zip"] = data["Pt Zip"].astype(str)

# Keep "Missing" values unchanged while trimming others to 5 digits
data["Pt Zip"] = data["Pt Zip"].apply(lambda x: x if x == "Missing" else str(x)[:5])

# Replace any blanks or 'nan' values with "Missing"
data["Pt Zip"] = data["Pt Zip"].replace(["nan", ""], "Missing").fillna("Missing")

# Capitalize only the first letter of each word
data["Language"] = data["Language"].str.title()

# Fill missing values with "Missing"
data["Language"] = data["Language"].fillna("Missing")

# Fix date of birth error
data["DOB"] = data["DOB"].replace("1/9/2973", "1/9/1973")

# Convert all values to strings
data["DOB"] = data["DOB"].astype(str)

# Standardize "Missing" entries
data["DOB"] = data["DOB"].replace(["MISSING", "nan"], "Missing")

# Convert valid dates to MM/DD/YYYY format while keeping "Missing" unchanged
data["DOB"] = data["DOB"].apply(lambda x: pd.to_datetime(x, errors="coerce"))
data["DOB"] = data["DOB"].apply(lambda x: x.strftime("%m/%d/%Y") if pd.notna(x) else "Missing")

# Correct spelling mistake
data["Marital Status"] = data["Marital Status"].replace("Seperated", "Separated")

# Capitalize only the first letter of each word
data["Marital Status"] = data["Marital Status"].str.title()

# Fill missing values with "Missing"
data["Marital Status"] = data["Marital Status"].fillna("Missing")

# Fill missing values with "Missing"
data["Gender"] = data["Gender"].fillna("Missing")

# Capitalize only the first letter of each word
data["Gender"] = data["Gender"].str.title()

# Strip unnecessary spaces
data["Gender"] = data["Gender"].str.strip()

# Correct misspellings
data["Race"] = data["Race"].replace({
    "American Indian or Alaskan Native": "American Indian or Alaska Native",
    "American Indian or Alaksa Native": "American Indian or Alaska Native",
    "Whiate": "White"
})

# Capitalize only the first letter of each word
data["Race"] = data["Race"].str.title()

# Fill missing values with "Missing"
data["Race"] = data["Race"].fillna("Missing")

# Correct misspellings and standardize values
data["Hispanic/Latino"] = data["Hispanic/Latino"].replace({
    "Hispanic Of Latino": "Hispanic or Latino",
    "Non-Hispanic Latino": "Non-Hispanic or Latino",
    "Non-Hispanic": "Non-Hispanic or Latino",
    "No": "Non-Hispanic or Latino",
    "Yes": "Hispanic or Latino"
})

# Strip unnecessary spaces
data["Hispanic/Latino"] = data["Hispanic/Latino"].str.strip()

# Capitalize first letter of each word
data["Hispanic/Latino"] = data["Hispanic/Latino"].str.title()

# Fill missing values with "Missing"
data["Hispanic/Latino"] = data["Hispanic/Latino"].fillna("Missing")

# Capitalize first letter of each word
data["Sexual Orientation"] = data["Sexual Orientation"].str.title()

# Correct misspellings and standardize values
data["Sexual Orientation"] = data["Sexual Orientation"].replace({
    "Decline": "Decline To Answer",
    "Female": "Missing",
    "Male": "Missing",
    "Heterosexual": "Straight",
    "Staight": "Straight",
    "Striaght": "Straight",
    "Stright": "Straight",
    "N/A": "Missing"
})

# Fill missing values with "Missing"
data["Sexual Orientation"] = data["Sexual Orientation"].fillna("Missing")

# Strip unnecessary spaces
data["Sexual Orientation"] = data["Sexual Orientation"].str.strip()

# Fill missing values with "Missing"
data["Insurance Type"] = data["Insurance Type"].fillna("Missing")

# Strip unnecessary spaces
data["Insurance Type"] = data["Insurance Type"].str.strip()

# Capitalize first letter of each word
data["Insurance Type"] = data["Insurance Type"].str.title()

# Correct misspellings and standardize values
data["Insurance Type"] = data["Insurance Type"].replace({
    "Medicaid & Medicare": "Medicare & Medicaid",
    "Uninsurred": "Uninsured",
    "Unisured": "Uninsured",
    "Unknown": "Missing"
})

# Strip spaces
data["Household Size"] = data["Household Size"].astype(str).str.strip()

# Standardize missing values
data["Household Size"] = data["Household Size"].replace(["nan", "Missing", "missing"], "Missing").fillna("Missing")

# Convert valid numbers to integers while keeping "Missing" unchanged
data["Household Size"] = data["Household Size"].apply(lambda x: int(x) if x.isnumeric() else x)

# Rid of outlier
data["Household Size"] = data["Household Size"].replace(4602, "Missing")

# Strip spaces
data["Total Household Gross Monthly Income"] = data["Total Household Gross Monthly Income"].astype(str).str.strip()

# Fill blanks with "Missing"
data["Total Household Gross Monthly Income"] = data["Total Household Gross Monthly Income"].replace(["nan", ""], "Missing").fillna("Missing")

# Format numeric values as monetary with two decimals while keeping "Missing" unchanged
data["Total Household Gross Monthly Income"] = data["Total Household Gross Monthly Income"].apply(lambda x: f"{float(x):,.2f}" if x.replace(".", "").isdigit() else x)

# Strip spaces
data["Distance roundtrip/Tx"] = data["Distance roundtrip/Tx"].astype(str).str.strip()

# Convert non-numeric values and blanks to "Missing"
data["Distance roundtrip/Tx"] = data["Distance roundtrip/Tx"].apply(lambda x: x if x.replace(".", "").isdigit() else "Missing")

# Capitalize first letter of each word
data["Referral Source"] = data["Referral Source"].str.title()

# Remove trailing spaces
data["Referral Source"] = data["Referral Source"].str.strip()

# Fill missing values with "Missing"
data["Referral Source"] = data["Referral Source"].fillna("Missing")

# Fill missing values with "Missing"
data["Referred By:"] = data["Referred By:"].fillna("Missing")

# Fill missing values with "Missing"
data["Amount"] = data["Amount"].fillna("Missing")

# Convert values that look like dates into "Yes"
data["Patient Letter Notified? (Directly/Indirectly through rep)"] = data["Patient Letter Notified? (Directly/Indirectly through rep)"].apply(
    lambda x: "Yes" if pd.to_datetime(x, errors="coerce") not in [None, pd.NaT] else x
)

# Capitalize first letter of each word
data["Patient Letter Notified? (Directly/Indirectly through rep)"] = data["Patient Letter Notified? (Directly/Indirectly through rep)"].str.title()

# Replace specific missing indicators with "Missing"
data["Patient Letter Notified? (Directly/Indirectly through rep)"] = data["Patient Letter Notified? (Directly/Indirectly through rep)"].replace(["", " ","na", "Na", "Hold", "nan", "HOLD"], "Missing").fillna("Missing")

# Remove trailing spaces
data["Patient Letter Notified? (Directly/Indirectly through rep)"] = data["Patient Letter Notified? (Directly/Indirectly through rep)"].str.strip()

# Strip spaces
data["Application Signed?"] = data["Application Signed?"].astype(str).str.strip()

# Capitalize first letter of each word
data["Application Signed?"] = data["Application Signed?"].str.title()

# Replace "N/A" and blanks with "Missing"
data["Application Signed?"] = data["Application Signed?"].replace(["", "N/A", "Nan"], "Missing").fillna("Missing")


['Missing' 'Yes' 'No']


In [ ]:
# Save cleaned dataset to a new Excel file
cleaned_file_path = r"C:\Users\njrav\Downloads\cleaned_data.xlsx"  # Adjust path as needed
data.to_excel(cleaned_file_path, index=False, engine="openpyxl")

In [83]:
import streamlit as st
import pandas as pd

# Cleaned file path
cleaned_file_path = r"C:\Users\njrav\Downloads\cleaned_data.xlsx"

# Load the cleaned data
@st.cache_data
def load_data():
    return pd.read_excel(cleaned_file_path)

data = load_data()

# Display full dataset for verification
st.title("Nebraska Cancer Specialists Hope Foundation Dashboard")
st.write("Below is the latest cleaned dataset:")
st.dataframe(data)

# Filter applications that are "Ready for Review" (i.e., "Request Status" == "Pending")
ready_for_review = data[data["Request Status"] == "Pending"]

st.subheader("Applications Ready for Review")
st.dataframe(ready_for_review)

# Dropdown to filter by "Application Signed?"
signed_status = st.selectbox("Filter by Application Signed Status", ["All", "Yes", "No", "Missing"])

if signed_status != "All":
    filtered_data = ready_for_review[ready_for_review["Application Signed?"] == signed_status]
else:
    filtered_data = ready_for_review

st.subheader("Filtered Applications")
st.dataframe(filtered_data)

2025-05-03 12:48:28.203 No runtime found, using MemoryCacheStorageManager
2025-05-03 12:48:30.814 Serialization of dataframe to Arrow table was unsuccessful due to: ("Expected bytes, got a 'int' object", 'Conversion failed for column Household Size with type object'). Applying automatic fixes for column types to make the dataframe Arrow-compatible.
2025-05-03 12:48:30.851 Serialization of dataframe to Arrow table was unsuccessful due to: ("Expected bytes, got a 'int' object", 'Conversion failed for column Household Size with type object'). Applying automatic fixes for column types to make the dataframe Arrow-compatible.
2025-05-03 12:48:30.901 Session state does not function when running a script without `streamlit run`
2025-05-03 12:48:30.918 Serialization of dataframe to Arrow table was unsuccessful due to: ("Expected bytes, got a 'int' object", 'Conversion failed for column Household Size with type object'). Applying automatic fixes for column types to make the dataframe Arrow-compa

DeltaGenerator()

In [84]:
streamlit run dashboard.py

SyntaxError: invalid syntax (512221001.py, line 1)